In [1]:
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn import datasets
import numpy as np

Using TensorFlow backend.


In [2]:
from google.colab import files
uploaded = files.upload()

Saving up_down_only.xlsx to up_down_only.xlsx


In [0]:
import io
df = pd.read_excel(io.BytesIO(uploaded['up_down_only.xlsx']), sep='\t', encoding = 'utf-8')
df = pd.DataFrame(df)

Equal up and down observations code, ONLY if necessary

In [0]:
df_down = df.iloc[0:93,]
#df_not_diff = df.iloc[93:351,].sample(n=93)
df_up = df.iloc[93:223,].sample(n=93)
df_93_counts = pd.concat([df_down, df_up]).sample(frac=1)
df=df_93_counts

93 counts code ends

In [0]:
df = df.sample(frac=1)

In [0]:
'''Shit Held code to remove one feature'''
try:
  df = df.drop(columns='denovo_ubx')
except:
  print ("Not Permissible")

Execute only to pass complete dataset

In [0]:
A = df.iloc[:,1:49]
B = df.iloc[:,49]

encoder = LabelEncoder()
encoder.fit(B)
encoded_b = encoder.transform(B)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_b = np_utils.to_categorical(encoded_b)


Complete dataset part ends

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

Stratified split, balanced split

In [0]:
InputX = df.iloc[:,1:49]
InputY = df.iloc[:,49]

A1, A2, B1, B2 = train_test_split(InputX, InputY, test_size=0.25, stratify=InputY)

encoder = LabelEncoder()
encoder.fit(B1)
encoded_b1 = encoder.transform(B1)
dummy_b1 = np_utils.to_categorical(encoded_b1)

Stratified split ends

*Unstratified* split starts

In [0]:
'''train1, test1 = train_test_split(df, test_size=0.25)

A1 = train1.iloc[:,1:49]
B1 = train1.iloc[:,49]
A2 = test1.iloc[:,1:49]
B2 = test1.iloc[:,49]


encoder = LabelEncoder()
encoder.fit(B1)
encoded_b1 = encoder.transform(B1)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_b1 = np_utils.to_categorical(encoded_b1)
'''

'train1, test1 = train_test_split(df, test_size=0.25)\n\nA1 = train1.iloc[:,1:49]\nB1 = train1.iloc[:,49]\nA2 = test1.iloc[:,1:49]\nB2 = test1.iloc[:,49]\n\n\nencoder = LabelEncoder()\nencoder.fit(B1)\nencoded_b1 = encoder.transform(B1)\n# convert integers to dummy variables (i.e. one hot encoded)\ndummy_b1 = np_utils.to_categorical(encoded_b1)\n'

Unstratified split ends

In [0]:
'''model = Sequential()
model.add(Dense(43, input_dim=48, activation='relu'))
model.add(Dense(38))
model.add(Dense(33))
model.add(Dense(28))
model.add(Dense(23))
model.add(Dense(18))
model.add(Dense(13))
model.add(Dense(8))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
test = model.fit(A1,dummy_b1, epochs = 50, batch_size = 5)'''

Hyperparameters adjustment based on thumb rules

In [53]:
model = Sequential()
model.add(Dense(43, input_dim=48, activation='relu'))
model.add(Dense(16))
model.add(Dense(8))
model.add(Dense(4))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
test = model.fit(A,dummy_b, epochs = 150, batch_size = 110) #batch size 10 optimal

Epoch 1/150
3/3 [==============================] - 0s 2ms/step - loss: 0.7766 - accuracy: 0.5541
Epoch 2/150
3/3 [==============================] - 0s 2ms/step - loss: 0.7358 - accuracy: 0.5856
Epoch 3/150
3/3 [==============================] - 0s 2ms/step - loss: 0.7259 - accuracy: 0.5766
Epoch 4/150
3/3 [==============================] - 0s 2ms/step - loss: 0.6551 - accuracy: 0.5676
Epoch 5/150
3/3 [==============================] - 0s 2ms/step - loss: 0.5552 - accuracy: 0.5721
Epoch 6/150
3/3 [==============================] - 0s 2ms/step - loss: 0.6607 - accuracy: 0.5856
Epoch 7/150
3/3 [==============================] - 0s 2ms/step - loss: 0.5682 - accuracy: 0.5946
Epoch 8/150
3/3 [==============================] - 0s 3ms/step - loss: 0.7518 - accuracy: 0.6036
Epoch 9/150
3/3 [==============================] - 0s 2ms/step - loss: 0.6234 - accuracy: 0.5991
Epoch 10/150
3/3 [==============================] - 0s 2ms/step - loss: 0.6651 - accuracy: 0.5901
Epoch 11/150
3/3 [===========

In [0]:
dff = pd.DataFrame(model.predict(A2))
dff.index = A2.index
#dff.rename(columns={0:'Down',1:'Not_diff',2:'up'}, inplace=True)
dff.rename(columns={0:'Down',1:'up'}, inplace=True)
result = dff.idxmax(axis=1)
result.index = dff.index

Removal of ambiguous results

In [0]:
#print(dff[abs(dff.diff(axis=1)['up'])>0.4])
b2df = B2[abs(dff.diff(axis=1)['up'])>0.1]
r1 = result[abs(dff.diff(axis=1)['up'])>0.1]

In [58]:
print('Removed ambigious results')
print(confusion_matrix(b2df, r1))
print(classification_report(b2df, r1))

print('All results')
print(confusion_matrix(B2, result))
print(classification_report(B2, result))

Removed ambigious results
[[20  2]
 [ 1 32]]
              precision    recall  f1-score   support

        Down       0.95      0.91      0.93        22
          up       0.94      0.97      0.96        33

    accuracy                           0.95        55
   macro avg       0.95      0.94      0.94        55
weighted avg       0.95      0.95      0.95        55

All results
[[21  2]
 [ 1 32]]
              precision    recall  f1-score   support

        Down       0.95      0.91      0.93        23
          up       0.94      0.97      0.96        33

    accuracy                           0.95        56
   macro avg       0.95      0.94      0.94        56
weighted avg       0.95      0.95      0.95        56



################
> Tracking Validation loss via plots

In [0]:
import matplotlib.pyplot as plt
plt.plot(test.history['acc'])
plt.plot(test.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [0]:
# summarize history for loss
plt.plot(test.history['loss'])
plt.plot(test.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

Split for 93 counts

In [0]:
df_down = df.iloc[0:93,]
df_not_diff = df.iloc[93:351,].sample(n=93)
df_up = df.iloc[351:480,].sample(n=93)
df_93_counts = pd.concat([df_down, df_not_diff, df_up]).sample(frac=1)
df=df_93_counts